In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# uv add rich

In [2]:
# 출력 예쁘게 하기
from rich.console import Console
from rich.table import Table

console = Console()

def rich_docs(docs, max_len=140, title="Retriever Results"):
    table = Table(title=title)
    table.add_column("#", justify="right")
    table.add_column("Source")
    table.add_column("Page", justify="right")
    table.add_column("Preview")

    for i, d in enumerate(docs, 1):
        m = d.metadata or {}
        src = (m.get("source","") or "").split("/")[-1]
        page = str(m.get("page_label", m.get("page",0)+1))
        text = (d.page_content or "").strip().replace("\n", " ")
        table.add_row(str(i), src, page, (text[:max_len] + ("…" if len(text) > max_len else "")))

    console.print(table)

### retriever 설정값
- 일반 RAG 기본값 : similarity or mmr
- 중복이 많을 경우 : mmr
- 그 외 필터링이 필요한 경우 : search_kwargs 에 다양한 옵션값을 넣어주면 됩니다

### advanced_retriever
- 길이가 길 경우 : compressed_retriever -> 필요할 때 Parent-Child
- 용어가 중요할 경우 : hybrid(vec + bm25) -> 부동산 관련 법률 용어
- 정확도 극대화 : similarity -> reranker -> reorder 

In [4]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma

In [7]:
emb = OpenAIEmbeddings(model = "text-embedding-3-small")
db_path = "../07_vectorstore/chroma_store"
col_name = "samsung_all"
vecstore = Chroma(
    persist_directory=db_path,
    collection_name=col_name,
    embedding_function=emb
)
vecstore._collection.count()

444

In [8]:
dim_size = emb.embed_query("안녕하세요")
print(len(dim_size))

1536


In [9]:
dim_size

[-0.002531265141442418,
 -0.06127675995230675,
 -0.008443817496299744,
 0.031540773808956146,
 0.031089577823877335,
 -0.04993239790201187,
 -0.059171177446842194,
 0.03244316577911377,
 -0.014116000384092331,
 -0.06140567362308502,
 -0.020722804591059685,
 0.006832401733845472,
 0.009195811115205288,
 -0.020067494362592697,
 -0.011484021320939064,
 0.035214800387620926,
 -0.04653768241405487,
 0.004017795901745558,
 -0.0141482288017869,
 0.027394063770771027,
 0.05706559494137764,
 -0.017306603491306305,
 -0.031368888914585114,
 -0.020185666158795357,
 0.04413130134344101,
 0.06239400804042816,
 0.056936681270599365,
 0.0005176672129891813,
 0.017166946083307266,
 -0.05358493700623512,
 0.0314333438873291,
 -0.02758743427693844,
 -0.01743551529943943,
 0.0011414192849770188,
 -0.002343266736716032,
 0.02537442371249199,
 0.02264576032757759,
 -0.04391644522547722,
 -0.011752590537071228,
 -0.034204982221126556,
 -0.013546633534133434,
 -0.018176767975091934,
 -0.00301334704272449,
 0.

### 1. 벡터 기반 검색기(유사도/mmr/score_threshold/filter)

In [21]:
question = "삼성의 지속 가능성에 대해 알려줘"
# 1. 유사도 기반
ret_sim = vecstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k" : 5}
)
result = ret_sim.invoke(question)
result

[Document(id='samsung_2025::1a74dfdc-0a31-4acf-9bd1-3fa7098f96e0', metadata={'creationdate': '2025-07-10T16:11:16+09:00', 'page': 85, 'total_pages': 87, 'trapped': '/False', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'source': '../00_data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'producer': 'Adobe PDF Library 15.0', 'page_label': '86', 'moddate': '2025-09-04T16:51:11+09:00'}, page_content='·  삼성전자주식회사 지속가능경영 웹사이트   \nhttp://www.samsung.com/sec/sustainability/main\n·  삼성전자주식회사 IR 웹사이트   \nhttp://www.samsung.com/sec/ir\n·  삼성전자주식회사 뉴스룸   \nhttp://news.samsung.com/kr   \nhttp://news.samsung.com/global\n담당 부서\n· 삼성전자주식회사 지속가능경영추진센터\n· 주소: 16677 경기도 수원시 영통구 삼성로 129(매탄동)\n· 이메일: sustainability.sec@samsung.com\n참고 자료\n·  사업 보고서 \n·  기업지배구조 보고서 \n·  삼성전자 책임광물 관리 보고서 \n·  행동규범 \n·  행동규범 가이드라인 \n미래 예측 진술 공지\n삼성전자주식회사의 지속가능경영보고서에서 삼성전자의 지속가능경영 목표 및  전략과 관련된 \n것을 포함하여 이 루어진 모 든 특정 내용은 관련법상 미래  예측 진술에 해 당할 수 있습니다. 이 \n지속가능경영보고서에서는 향후 상황 및 지속가능경영 성과에 대한 삼성전자의 현재 견해를 반영하는 \n미래 예측 진

In [19]:
rich_docs(result, title="유사도 기반 탑 5개 확인")

                                              유사도 기반 탑 5개 확인                                              
┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ # ┃ Source                                           ┃ Page ┃ Preview                                           ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1 │ Samsung_Electronics_Sustainability_Report_2025_… │   86 │ ·  삼성전자주식회사 지속가능경영 웹사이트         │
│   │                                                  │      │ http://www.samsung.com/sec/sustainability/main ·  │
│   │                                                  │      │ 삼성전자주식회사 IR 웹사이트                      │
│   │                                                  │      │ http://www.samsung.com/sec/ir ·  삼성전자주식회 … │
│   │                                                  │      │ 뉴…                                               │
│ 2 │ Sustainability_report_2024_kr.pdf                │   82 │ 비록 삼성전자주식회사는 지속가능경영보고서의 미 … │
│   │                                                  │      │ 예측 진술이 시의성 있고 합리적인 정보, 가정  및   │
│   │                                                  │      │ 믿음에 기반한다고 판단하지만, 이러한 미래 예측    │
│   │                                                  │      │ 진술(그리고 이를 이루는 정보, 가정 및 믿음)은     │
│   │                                                  │      │ 다양한 요인, 리스크, 불확실성의 영향권에          │
│   │                                                  │      │ 있으므로…                                         │
│ 3 │ Sustainability_report_2024_kr.pdf                │   82 │ ISAE3000을 적용했습니다. 관련 정보 ·              │
│   │                                                  │      │ 삼성전자주식회사 대표 홈페이지                    │
│   │                                                  │      │ http://www.samsung.com/sec ·  삼성전자주식회사    │
│   │                                                  │      │ 지속가능경영 웹사이트                             │
│   │                                                  │      │ http://www.samsung.com/sec/sustainabilit…         │
│ 4 │ Samsung_Electronics_Sustainability_Report_2025_… │   86 │ 삼성전자 지속가능경영보고서 2025 86               │
│   │                                                  │      │ 삼성전자주식회사는 경제·사회·환경적 가치 창출     │
│   │                                                  │      │ 성과를 다양한 이해관계자와 투명하게 소통하기 위 … │
│   │                                                  │      │ 2025년 열여덟 번째 지속가능경영보고서를           │
│   │                                                  │      │ 발간합니다. 작성 기준 본 보고서는 지속가능경영    │
│   │                                                  │      │ 보고 기준인 GRI(G…                                │
│ 5 │ Samsung_Electronics_Sustainability_Report_2025_… │   86 │ 비록 삼성전자주식회사는 지속가능경영보고서의 미 … │
│   │                                                  │      │ 예측 진술이 시의성 있고 합리적인 정보, 가정  및   │
│   │                                                  │      │ 믿음에 기반한다고 판단하지만, 이러한 미래 예측    │
│   │                                                  │      │ 진술(그리고 이를 이루는 정보, 가정 및 믿음)은     │
│   │                                                  │      │ 다양한 요인, 리스크, 불확실성의 영향권에          │
│   │                                                  │      │ 있으므로…                                         │
└───┴──────────────────────────────────────────────────┴──────┴───────────────────────────────────────────────────┘

In [20]:
ret_mmr = vecstore.as_retriever(
    search_type = "mmr",
    search_kwargs = {"k" : 5,
                     "fetch_k" : 20,
                     "lambda_mult" : 0.5
                     }
)
result1 = ret_sim.invoke(question)
rich_docs(result1, title="mmr 로 중복 최소화")

                                                mmr 로 중복 최소화                                                 
┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ # ┃ Source                                           ┃ Page ┃ Preview                                           ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1 │ Samsung_Electronics_Sustainability_Report_2025_… │   86 │ ·  삼성전자주식회사 지속가능경영 웹사이트         │
│   │                                                  │      │ http://www.samsung.com/sec/sustainability/main ·  │
│   │                                                  │      │ 삼성전자주식회사 IR 웹사이트                      │
│   │                                                  │      │ http://www.samsung.com/sec/ir ·  삼성전자주식회 … │
│   │                                                  │      │ 뉴…                                               │
│ 2 │ Sustainability_report_2024_kr.pdf                │   82 │ 비록 삼성전자주식회사는 지속가능경영보고서의 미 … │
│   │                                                  │      │ 예측 진술이 시의성 있고 합리적인 정보, 가정  및   │
│   │                                                  │      │ 믿음에 기반한다고 판단하지만, 이러한 미래 예측    │
│   │                                                  │      │ 진술(그리고 이를 이루는 정보, 가정 및 믿음)은     │
│   │                                                  │      │ 다양한 요인, 리스크, 불확실성의 영향권에          │
│   │                                                  │      │ 있으므로…                                         │
│ 3 │ Sustainability_report_2024_kr.pdf                │   82 │ ISAE3000을 적용했습니다. 관련 정보 ·              │
│   │                                                  │      │ 삼성전자주식회사 대표 홈페이지                    │
│   │                                                  │      │ http://www.samsung.com/sec ·  삼성전자주식회사    │
│   │                                                  │      │ 지속가능경영 웹사이트                             │
│   │                                                  │      │ http://www.samsung.com/sec/sustainabilit…         │
│ 4 │ Samsung_Electronics_Sustainability_Report_2025_… │   86 │ 삼성전자 지속가능경영보고서 2025 86               │
│   │                                                  │      │ 삼성전자주식회사는 경제·사회·환경적 가치 창출     │
│   │                                                  │      │ 성과를 다양한 이해관계자와 투명하게 소통하기 위 … │
│   │                                                  │      │ 2025년 열여덟 번째 지속가능경영보고서를           │
│   │                                                  │      │ 발간합니다. 작성 기준 본 보고서는 지속가능경영    │
│   │                                                  │      │ 보고 기준인 GRI(G…                                │
│ 5 │ Samsung_Electronics_Sustainability_Report_2025_… │   86 │ 비록 삼성전자주식회사는 지속가능경영보고서의 미 … │
│   │                                                  │      │ 예측 진술이 시의성 있고 합리적인 정보, 가정  및   │
│   │                                                  │      │ 믿음에 기반한다고 판단하지만, 이러한 미래 예측    │
│   │                                                  │      │ 진술(그리고 이를 이루는 정보, 가정 및 믿음)은     │
│   │                                                  │      │ 다양한 요인, 리스크, 불확실성의 영향권에          │
│   │                                                  │      │ 있으므로…                                         │
└───┴──────────────────────────────────────────────────┴──────┴───────────────────────────────────────────────────┘

In [23]:
# 3. score 로 제어 - threshold 값 지정
ret_score = vecstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.4,
                   "k" : 8,
                   },
)
result = ret_score.invoke(question)
rich_docs(result, title="score 로 확인")

                                                   score 로 확인                                                   
┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ # ┃ Source                                           ┃ Page ┃ Preview                                           ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1 │ Samsung_Electronics_Sustainability_Report_2025_… │   86 │ ·  삼성전자주식회사 지속가능경영 웹사이트         │
│   │                                                  │      │ http://www.samsung.com/sec/sustainability/main ·  │
│   │                                                  │      │ 삼성전자주식회사 IR 웹사이트                      │
│   │                                                  │      │ http://www.samsung.com/sec/ir ·  삼성전자주식회 … │
│   │                                                  │      │ 뉴…                                               │
└───┴──────────────────────────────────────────────────┴──────┴───────────────────────────────────────────────────┘

In [24]:
vecstore._collection.get(limit=1)

{'ids': ['samsung_2024::a4e1d10d-eba2-4c8b-b9b2-b1d2a279899d'],
 'embeddings': None,
 'documents': ['A Journey Towards  \na Sustainable Future\n삼성전자 지속가능경영보고서 2024'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'producer': 'Adobe PDF Library 15.0',
   'moddate': '2024-11-25T11:10:46+09:00',
   'trapped': '/False',
   'total_pages': 83,
   'page_label': '1',
   'creationdate': '2024-11-25T11:10:32+09:00',
   'source': '../00_data/Sustainability_report_2024_kr.pdf',
   'page': 0,
   'creator': 'Adobe InDesign 15.1 (Macintosh)'}]}

In [27]:
# 3. score 로 제어 - threshold 값 지정
ret_score = vecstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.2,
                   "k" : 8,
                   "filter": {"source": "../00_data/Sustainability_report_2024_kr.pdf"}
                   },
)
result = ret_score.invoke(question)
rich_docs(result, title="score 로 확인")

                                                   score 로 확인                                                   
┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ # ┃ Source                            ┃ Page ┃ Preview                                                          ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1 │ Sustainability_report_2024_kr.pdf │   82 │ 비록 삼성전자주식회사는 지속가능경영보고서의 미래 예측 진술이    │
│   │                                   │      │ 시의성 있고 합리적인 정보, 가정  및 믿음에 기반한다고            │
│   │                                   │      │ 판단하지만, 이러한 미래 예측 진술(그리고 이를 이루는 정보, 가정  │
│   │                                   │      │ 및 믿음)은  다양한 요인, 리스크, 불확실성의 영향권에 있으므로…   │
│ 2 │ Sustainability_report_2024_kr.pdf │   82 │ ISAE3000을 적용했습니다. 관련 정보 ·  삼성전자주식회사 대표      │
│   │                                   │      │ 홈페이지   http://www.samsung.com/sec ·  삼성전자주식회사        │
│   │                                   │      │ 지속가능경영 웹사이트                                            │
│   │                                   │      │ http://www.samsung.com/sec/sustainabilit…                        │
│ 3 │ Sustainability_report_2024_kr.pdf │   55 │ 삼성전자 지속가능경영보고서 2024 55Our Company                   │
│   │                                   │      │ AppendixMateriality Assessment Facts & Figures PrinciplePlanet   │
│   │                                   │      │ People Facts & Figures 56     경제성과 57     사…                │
│ 4 │ Sustainability_report_2024_kr.pdf │   30 │ 삼성전자 지속가능경영보고서 2024 30Our Company                   │
│   │                                   │      │ AppendixMateriality Assessment Facts & Figures PrinciplePlanet   │
│   │                                   │      │ People 사회적 책임을 다하며 미래로 함께 나아갑니다. People 31 …  │
│ 5 │ Sustainability_report_2024_kr.pdf │    4 │ 삼성전자 지속가능경영보고서 2024 04 Our Company                  │
│   │                                   │      │ AppendixMateriality Assessment Facts & Figures PrinciplePlanet   │
│   │                                   │      │ People CEO 메시지 Message from  Our CEO 주주, 고…                │
│ 6 │ Sustainability_report_2024_kr.pdf │   82 │ "outlook," "believes," "estimates," "predicts," "potential,"     │
│   │                                   │      │ "continue," " preliminary,"   “추진”, “목표”, “전략”, “예정”과   │
│   │                                   │      │ 같은 용어로 미래 예측 진술을 식별할 수 있습…                     │
│ 7 │ Sustainability_report_2024_kr.pdf │   82 │ 삼성전자 지속가능경영보고서 2024 82 Our Company                  │
│   │                                   │      │ AppendixMateriality Assessment Facts & Figures PrinciplePlanet   │
│   │                                   │      │ People About This Report 삼성전자주식회사는 경제·사회·환…        │
│ 8 │ Sustainability_report_2024_kr.pdf │   52 │ 삼성전자 지속가능경영보고서 2024 52Our Company                   │
│   │                                   │      │ AppendixMateriality Assessment Facts & Figures PrinciplePlanet   │
│   │                                   │      │ People 지속가능한 미래를 위해 책임경영을 실천합니다. Principle … │
└───┴───────────────────────────────────┴──────┴──────────────────────────────────────────────────────────────────┘